In [6]:
!pip install alpha_vantage

In [9]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
import time

def get_financial_data_robust(tickers, delay_between_tickers=0.25):
    """
    Fetches and calculates financial data based on the Flourish Explorer requirements,
    using the robust yfinance.download() method.

    This method can calculate all Performance and Trading metrics, but cannot fetch
    fundamental data like Sector, Industry, or P/E Ratio.

    Args:
        tickers (list): A list of stock ticker symbols.
        delay_between_tickers (float): A small delay to be a good citizen.

    Returns:
        pandas.DataFrame: A DataFrame containing the structured financial data.
    """
    all_data = []
    print(f"Starting robust data extraction for {len(tickers)} tickers...\n")

    for i, symbol in enumerate(tickers):
        print(f"  ({i+1}/{len(tickers)}) Processing {symbol}...")

        try:
            # Download 13 months of data to ensure we have a full year for calculations
            df = yf.download(symbol, period="13mo", interval="1d", progress=False, auto_adjust=True)

            if df.empty:
                print(f"    - WARNING: No data found for {symbol}. Skipping.\n")
                continue

            # --- Extract and Calculate Metrics from the DataFrame ---

            # Basic & Trading Metrics
            last_close = df['Close'].iloc[-1]
            prev_close = df['Close'].iloc[-2] if len(df) > 1 else last_close
            price_change = last_close - prev_close
            percent_change = (price_change / prev_close) * 100 if prev_close != 0 else 0
            avg_volume = df['Volume'].tail(63).mean() # ~3 months average volume

            # Valuation Metrics (Calculable ones)
            # Use the last 252 trading days (~1 year) for 52-week high/low
            last_year_data = df.tail(252)
            fifty_two_week_high = last_year_data['High'].max()
            fifty_two_week_low = last_year_data['Low'].min()

            # Performance Metrics
            daily_returns = df['Close'].pct_change().dropna()
            
            ytd_start_price = df[df.index.year == datetime.now().year]['Close'].iloc[0] if not df[df.index.year == datetime.now().year].empty else None
            ytd_return = (last_close / ytd_start_price - 1) * 100 if ytd_start_price else np.nan
            
            # Helper function for safe return calculation
            def safe_return(days):
                return (df['Close'].iloc[-1] / df['Close'].iloc[-days] - 1) * 100 if len(df) >= days else np.nan

            one_month_return = safe_return(22)  # ~22 trading days in a month
            three_month_return = safe_return(63) # ~63 trading days in a quarter
            one_year_return = safe_return(252) # ~252 trading days in a year
            
            volatility = np.std(daily_returns.values) * np.sqrt(252) * 100 # Annualized volatility in %

            # --- Assemble the dictionary for this ticker ---
            data = {
                'Symbol': symbol,
                # --- Basic Info ---
                'Current Price': last_close,
                # --- Valuation Metrics (Calculable) ---
                '52W High': fifty_two_week_high,
                '52W Low': fifty_two_week_low,
                # --- Performance Metrics ---
                'YTD Return': ytd_return,
                '1M Return': one_month_return,
                '3M Return': three_month_return,
                '1Y Return': one_year_return,
                'Volatility (Ann.)': volatility,
                # --- Trading Metrics ---
                'Avg Volume (3M)': avg_volume,
                'Price Change': price_change,
                'Percent Change': percent_change,
            }

            all_data.append(data)
            print(f"    + SUCCESS: Data fetched for {symbol}.\n")

        except Exception as e:
            print(f"    - ERROR: An unexpected error occurred for {symbol}: {e}\n")

        time.sleep(delay_between_tickers)

    if not all_data:
        print("\nNo data was successfully extracted.")
        return pd.DataFrame()

    # Create and format the final DataFrame
    df_final = pd.DataFrame(all_data).set_index('Symbol')

    # Format percentage columns for clarity
    pct_cols = ['YTD Return', '1M Return', '3M Return', '1Y Return', 'Volatility (Ann.)', 'Percent Change']
    for col in pct_cols:
        df_final[col] = pd.to_numeric(df_final[col], errors='coerce').apply(lambda x: f"{x:.2f}%" if pd.notna(x) else 'N/A')

    # Format large number columns
    df_final['Avg Volume (3M)'] = df_final['Avg Volume (3M)'].apply(lambda x: f"{x:,.0f}" if pd.notna(x) else 'N/A')

    print("\nData extraction complete.")
    return df_final


# --- Example Usage ---
if __name__ == "__main__":
    tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'JPM', 'NVDA', 'NONEXISTENTTICKER123']
    df_summary = get_financial_data_robust(tickers)

    if not df_summary.empty:
        print("\n--- Flourish Explorer: Financial Data Summary ---")
        pd.set_option('display.max_columns', None)
        print(df_summary)

        # Optional: Save to CSV
        # df_summary.to_csv("flourish_explorer_summary.csv")
        # print("\nSaved to flourish_explorer_summary.csv")
        

Starting robust data extraction for 8 tickers...

  (1/8) Processing AAPL...



1 Failed download:
['AAPL']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


    - WARNING: No data found for AAPL. Skipping.

  (2/8) Processing MSFT...



1 Failed download:
['MSFT']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


    - WARNING: No data found for MSFT. Skipping.

  (3/8) Processing GOOGL...


...

    - WARNING: No data found for JPM. Skipping.

  (7/8) Processing NVDA...



1 Failed download:
['NVDA']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


    - WARNING: No data found for NVDA. Skipping.

  (8/8) Processing NONEXISTENTTICKER123...



1 Failed download:
['NONEXISTENTTICKER123']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


    - WARNING: No data found for NONEXISTENTTICKER123. Skipping.


No data was successfully extracted.
